In [ ]:
# Import the relevant modules
import os
import sys
import glob

# Import third party modules
import numpy as np
import xarray as xr
import regionmask
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import dictionaries_em as dicts

In [ ]:
# # Load in the onshore wind power data
# files = glob.glob(
#     os.path.join(dicts.clearheads_dir, "NUTS_0_wp_ons_sim_0_historical_loc_weighted.nc")
# )

# # Load the file
# ds = xr.open_dataset(files[0], chunks={"time": 100})

# # Print tyhe dataset
# print(ds)

In [ ]:
# # Print the nuts key values
# print(ds.NUTS_keys.values)

In [ ]:
# nuts_keys = ds.NUTS_keys.values

In [ ]:
# # Load in the shapefile
# NUTS_shapefile = gpd.read_file("shapefiles/NUTS/NUTS_RG_10M_2021_4326.shp")

# # Look at the head of the file
# NUTS_shapefile.head()

In [ ]:
# # Using the dictionary for country codes, constrain the shapefile to only include those in the country codes
# NUTS_shapefile = NUTS_shapefile[NUTS_shapefile["CNTR_CODE"].isin(nuts_keys)]

# # Constrain to onlty national level
# NUTS_shapefile = NUTS_shapefile[NUTS_shapefile["LEVL_CODE"] == 0]

In [ ]:
# # Load in the shapefile fo the eez data
# EEZ_shapefile = gpd.read_file("shapefiles/EEZ/eez_v12.shp")

# # Constrain the EEZ shapefile to where the column "SOVEREIGN2" is equal
# # to the country lists in the dictionary
# EEZ_shapefile = EEZ_shapefile[EEZ_shapefile["SOVEREIGN1"].isin(dicts.countries_list)]

In [ ]:
# EEZ_shapefile

In [ ]:
# # Print all of the column names for the eeZ shapefile
# print(EEZ_shapefile.columns)

In [ ]:
# # Plot these shapes with different colors for each country
# plt.figure(figsize=(10, 10))
# ax = plt.axes()
# NUTS_shapefile.boundary.plot(ax=ax)

In [ ]:
# # PLot the eez shapes
# plt.figure(figsize=(10, 10))
# ax = plt.axes()
# EEZ_shapefile.geometry.plot(ax=ax)
# # Constrain to specific bounds
# ax.set_xlim(-50, 50)
# ax.set_ylim(30, 80)

Now we want to process the correlation data for the EEZ domains, then plot the strength of these correlations on the map.

In [ ]:
# # Import the functions
# import functions_em as funcs

# # Extract the data from the .nc file
# eez_cfs = funcs.extract_offshore_eez_to_df(
#     filepath=os.path.join(dicts.clearheads_dir, "EEZ_zones_wp_historical.nc")
# )

In [ ]:
# # Look at the head of the dataframe
# eez_cfs.head()

In [ ]:
# Import the functions
import functions_em as funcs

In [ ]:
import importlib

importlib.reload(funcs)

In [ ]:
# test the other function for doing this
merged_df, corr_df = funcs.correlate_nao_uread(
    filename="EEZ_zones_wp_historical.nc",
    shp_file="eez_v12.shp",
    shp_file_dir="shapefiles/EEZ/",
    obs_var="si10",
)

In [ ]:
df_ts

In [ ]:
cfs.head()

In [ ]:
corr_df.head()

In [ ]:
# Drop the Nans from the dataframe
corr_df = corr_df.dropna()

In [ ]:
# Load in the shapefile fo the eez data
EEZ_shapefile = gpd.read_file("shapefiles/EEZ/eez_v12.shp")

In [ ]:
EEZ_shapefile.head()

In [ ]:
# Print all of the column names for the eeZ shapefile
print(EEZ_shapefile.columns)

In [ ]:
# Throw away all of the columns, apart from "GEONAME", 'SOVEREIGN1',
# "ISOSOV1", "geometry"
EEZ_shapefile = EEZ_shapefile[["GEONAME", "SOVEREIGN1", "ISO_SOV1", "geometry"]]

In [ ]:
EEZ_shapefile.head()

In [ ]:
iso_sov1 = EEZ_shapefile["ISO_SOV1"].values

In [ ]:
iso_sov1

In [ ]:
# Extract the values of the region column from corr_df
region_values = corr_df.region.values

In [ ]:
region_values

In [ ]:
# reload the dictionary
importlib.reload(dicts)

In [ ]:
# Convert the region values to equivalent iso_sov1 values
# using the mapping in the dictionary
iso_sov1_values = [dicts.iso_mapping[region] for region in region_values]

In [ ]:
iso_sov1_values

In [ ]:
# Constrain the geo dataframe to only include the iso_sov1 values
EEZ_shapefile = EEZ_shapefile[EEZ_shapefile["ISO_SOV1"].isin(iso_sov1_values)]

In [ ]:
# Find where ISO_SOV1 is equal to "ITA"
EEZ_shapefile.head()

In [ ]:
# Where corr_df.region passed through iso_mapping dict is
# equal to the values in EEZ_shapefile.ISO_SOV1
# Add the corresponding correlation and p-value to the dataframe

In [ ]:
# Filter df to only include the rows where GEONAME includes: "Exclusive Economic Zone"
EEZ_shapefile = EEZ_shapefile[
    EEZ_shapefile["GEONAME"].str.contains("Exclusive Economic Zone")
]

In [ ]:
EEZ_shapefile.head()

In [ ]:
# Now we want to append the correlation and p-value to the dataframe
# Add a new column to corr_df called "ISO_SOV1"
corr_df["ISO_SOV1"] = iso_sov1_values

In [ ]:
corr_df.head()

In [ ]:
# Loop over the columns in EEZ_shapefile and add the correlation and p-value
# where the ISO_SOV1 values are equal
for index, row in EEZ_shapefile.iterrows():
    # Extract the ISO_SOV1 value
    iso_sov1 = row["ISO_SOV1"]
    # Find the index of the row in corr_df that matches the ISO_SOV1
    index_corr = corr_df[corr_df["ISO_SOV1"] == iso_sov1].index
    # Add the correlation and p-value to the dataframe
    EEZ_shapefile.loc[index, "correlation"] = corr_df.loc[
        index_corr, "correlation"
    ].values
    EEZ_shapefile.loc[index, "p-value"] = corr_df.loc[index_corr, "p-value"].values

In [ ]:
EEZ_shapefile

In [ ]:
# Remove any rows from EEZ shapefile which contain "(*)" in the GEONAME column
EEZ_shapefile = EEZ_shapefile[~EEZ_shapefile["GEONAME"].str.contains(r"\(.*\)")]

In [ ]:
EEZ_shapefile

In [ ]:
print(type(EEZ_shapefile))

In [ ]:
# Reload the dicts
importlib.reload(dicts)

In [ ]:
# Import cartopy
import cartopy.crs as ccrs

# Now plot the EEZ_shapefile with the correlation as the color
plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
EEZ_shapefile.plot(
    column="correlation",
    ax=ax,
    legend=True,
    cmap="coolwarm",
    legend_kwds={
        "label": "Correlation",
        "orientation": "horizontal",
        "shrink": 0.8,
        "pad": 0.01,
    },
)
# Use cartopy to add the coastlines
ax.coastlines()
# Make the colorbar smaller
cbar = ax.get_figure().get_axes()[1]
cbar.set_ylabel("Correlation", fontsize=12)
cbar.tick_params(labelsize=10)

# Extract the lats of the northern eu grid box
lat1, lat2 = dicts.northern_europe_grid["lat1"], dicts.northern_europe_grid["lat2"]
lon1, lon2 = dicts.northern_europe_grid["lon1"], dicts.northern_europe_grid["lon2"]

# Plot the grid box
plt.plot([lon1, lon2, lon2, lon1, lon1], [lat1, lat1, lat2, lat2, lat1], "r")

# Include hazels grid box
lat1_n, lat2_n = dicts.uk_n_box_corrected["lat1"], dicts.uk_n_box_corrected["lat2"]
lon1_n, lon2_n = dicts.uk_n_box_corrected["lon1"], dicts.uk_n_box_corrected["lon2"]

# Plot the grid box
plt.plot(
    [lon1_n, lon2_n, lon2_n, lon1_n, lon1_n],
    [lat1_n, lat1_n, lat2_n, lat2_n, lat1_n],
    "g",
)

# Include hazels grid box
lat1_s, lat2_s = dicts.uk_s_box_corrected["lat1"], dicts.uk_s_box_corrected["lat2"]
lon1_s, lon2_s = dicts.uk_s_box_corrected["lon1"], dicts.uk_s_box_corrected["lon2"]

# Plot the grid box
plt.plot(
    [lon1_s, lon2_s, lon2_s, lon1_s, lon1_s],
    [lat1_s, lat1_s, lat2_s, lat2_s, lat1_s],
    "g",
)

# Include ticks for the lat and lon
ax.gridlines(draw_labels=True)

# Constrain to specific bounds
ax.set_xlim(-40, 50)
ax.set_ylim(30, 80)

In [ ]:
# Reload the dictionary
importlib.reload(dicts)

In [ ]:
# Limit the EEZ_shapefile to only include only the ISO_SOV1 values
# Which are in dicts.eez_agg_countries
EEZ_shapefile_n = EEZ_shapefile[EEZ_shapefile["ISO_SOV1"].isin(dicts.eez_agg_countries)]

In [ ]:
# Now plot the EEZ_shapefile with the correlation as the color
plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
EEZ_shapefile_n.plot(
    column="correlation",
    ax=ax,
    legend=True,
    cmap="coolwarm",
    legend_kwds={
        "label": "Correlation",
        "orientation": "horizontal",
        "shrink": 0.8,
        "pad": 0.01,
    },
)
# Use cartopy to add the coastlines
ax.coastlines()
# Make the colorbar smaller
cbar = ax.get_figure().get_axes()[1]
cbar.set_ylabel("Correlation", fontsize=12)
cbar.tick_params(labelsize=10)

# Include ticks for the lat and lon
ax.gridlines(draw_labels=True)

# Constrain to specific bounds
ax.set_xlim(-40, 50)
ax.set_ylim(30, 80)

In [ ]:
corr_df.head()

In [ ]:
cfs.head()

In [ ]:
# reload the dicts
importlib.reload(dicts)

In [ ]:
# replace the column headings in cfs with those mapped using dicts.iso_mapping
for key in dicts.iso_mapping:
    cfs = cfs.rename(columns={key: dicts.iso_mapping[key]})

In [ ]:
cfs.head()

In [ ]:
# Reload dicts
importlib.reload(dicts)

In [ ]:
# Create a new column in cfs called N_Europe
# which is the average of all of the countries (columns) in dicts.eez_agg_countries
cfs["N_Europe"] = cfs[dicts.eez_agg_countries].mean(axis=1)

In [ ]:
cfs.head()

In [ ]:
from scipy.stats import pearsonr

# Create a plot with two y-axes
# Time on the x-axes
# The variable on the left y-axes is the NAO anomaly (Pa)
# The variable on the right y-axes is the wind power (GW) for N_Europe
fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot the NAO anomaly
ax1.plot(cfs.index, cfs["NAO anomaly (Pa)"] / 100, "b-")

# Set the x-axis label
ax1.set_xlabel("Time")

# Set the y-axis label
ax1.set_ylabel("NAO anomaly (hPa)", color="b")

# Include a black dashed line for y=0
ax1.axhline(0, color="black", linestyle="--")

# Set the color of the ticks
ax1.tick_params("y", colors="b")

# Create a second y-axis
ax2 = ax1.twinx()

# Plot the wind power
ax2.plot(cfs.index, cfs.N_Europe, "r-")

# Set the y-axis label
ax2.set_ylabel("Wind Power (GW)", color="r")

# Set the colour of the ticks
ax2.tick_params("y", colors="r")

# Calculate the correlation between the NAO anomaly and the wind power
corr, p = pearsonr(cfs["NAO anomaly (Pa)"], cfs.N_Europe)

# Include the correlation and p-value on the plot
ax2.text(
    0.05,
    0.95,
    f"Correlation: {corr:.2f}\nP-value: {p:.2f}",
    horizontalalignment="left",
    verticalalignment="top",
    bbox=dict(facecolor="white", alpha=0.5),
    transform=ax2.transAxes,
)

# Show the plot
plt.show()

In [ ]:
# Plot a scatter plot of NAO agaist wind power
plt.figure(figsize=(8, 8))

# Plot the scatter plot
plt.scatter(cfs["NAO anomaly (Pa)"], cfs.N_Europe, color="k")

# Include a line of best fit
m, b = np.polyfit(cfs["NAO anomaly (Pa)"], cfs.N_Europe, 1)

# Plot the line of best fit
plt.plot(cfs["NAO anomaly (Pa)"], m * cfs["NAO anomaly (Pa)"] + b, "k")

# Set the x-axis label
plt.xlabel("Obs NAO anomaly (Pa)", color="b")

# Set the xticks to blue
plt.tick_params(axis="x", colors="b")

# Set the y-axis label
plt.ylabel("Obs Wind Power (GW)", color="r")

# Set the yticks to red
plt.tick_params(axis="y", colors="r")

In [ ]:
# Load in the ERA5 data for the NAO index
# Use this file
# adaptor.mars.internal-1691509121.3261805-29348-4-3a487c76-fc7b-421f-b5be-7436e2eb78d7.nc
# in ~/ERA5/
# Load the dataset
era5_data_path = "~/ERA5/adaptor.mars.internal-1691509121.3261805-29348-4-3a487c76-fc7b-421f-b5be-7436e2eb78d7.nc"

# Load the data into chunks
ds_era5 = xr.open_mfdataset(
    era5_data_path,
    combine="by_coords",
    parallel=True,
    chunks={"time": 100, "latitude": 100, "longitude": 100},
)[
    "msl"
]  # for mean sea level pressure

# Combine the first two expver variables
obs_msl = ds_era5.sel(expver=1).combine_first(ds_era5.sel(expver=5))

In [ ]:
# Constrain obs to ONDJFM
obs_msl = obs_msl.sel(time=obs_msl.time.dt.month.isin([10, 11, 12, 1, 2, 3]))

# Shift the time index back by 3 months
obs_msl_shifted = obs_msl.shift(time=-3)

# Take annual means
obs_msl_annual = obs_msl_shifted.resample(time="Y").mean()

# Throw away years 1959, 2021, 2022 and 2023
obs_msl_annual = obs_msl_annual.sel(time=slice("1960", "2019"))

# Remove the climatology
obs_msl_anomaly = obs_msl_annual - obs_msl_annual.mean(dim="time")

In [ ]:
# Extract the lats and lons of the azores
lat1, lat2 = dicts.era5_azores["lat1"], dicts.era5_azores["lat2"]
lon1, lon2 = dicts.era5_azores["lon1"], dicts.era5_azores["lon2"]

# Calculate the mean for the azores gridbox
obs_msl_anomaly_azores = obs_msl_anomaly.sel(
    latitude=slice(lat1, lat2), longitude=slice(lon1, lon2)
).mean(dim=["latitude", "longitude"])

In [ ]:
# Same for iceland
lat1, lat2 = dicts.era5_iceland["lat1"], dicts.era5_iceland["lat2"]
lon1, lon2 = dicts.era5_iceland["lon1"], dicts.era5_iceland["lon2"]

# Calculate the mean for the iceland gridbox
obs_msl_anomaly_iceland = obs_msl_anomaly.sel(
    latitude=slice(lat1, lat2), longitude=slice(lon1, lon2)
).mean(dim=["latitude", "longitude"])

In [ ]:
# Calculate the NAO index (azores - iceland)
nao_index = obs_msl_anomaly_azores - obs_msl_anomaly_iceland

In [ ]:
# EXtract the time series
nao_index_time = nao_index.time.values

# Extract the values
nao_index_values = nao_index.values

# Create a dataframe
nao_df = pd.DataFrame({"time": nao_index_time, "value": nao_index_values})

# Take a centred 8-year running mean
nao_running = nao_df.set_index("time").rolling(8, center=True).mean()

In [ ]:
# Have a look at the dataframe
nao_running.head()

In [ ]:
# Drop the NaN values
nao_running = nao_running.dropna()

In [ ]:
# Combine the two dataframes into one, using the index of the first
eez_df = eez_cfs.join(nao_running, how="inner")

In [ ]:
eez_df.head()

In [ ]:
# Rename the value column as 'NAO anomaly (Pa)'
eez_df = eez_df.rename(columns={"value": "NAO anomaly (Pa)"})

In [ ]:
# Drop the rows which contain NaN values in the NAO anomaly column
eez_df = eez_df.dropna()

In [ ]:
eez_df.head()

In [ ]:
from scipy.stats import pearsonr
import pandas as pd

# Create a new dataframe with columns for:
# 'region' - e.g. Netherlands_7
# 'correlation' - the correlation between the NAO and the offshore wind CFs
# 'p-value' - the p-value of the correlation
# Set up the dataframe
correlation_df = pd.DataFrame(columns=["region", "correlation", "p-value"])

# Loop over the regions
for region in eez_df.columns[:-1]:
    # Calculate the correlation
    corr, p = pearsonr(eez_df[region], eez_df["NAO anomaly (Pa)"])

    # Create a new DataFrame to append
    df_to_append = pd.DataFrame(
        {"region": [region], "correlation": [corr], "p-value": [p]}
    )

    # Append to the dataframe
    correlation_df = pd.concat([correlation_df, df_to_append], ignore_index=True)

In [ ]:
correlation_df.head()

In [ ]:
# Remove the numbers from the region column by removing the last 2 characters
correlation_df["region"] = correlation_df["region"].str

In [ ]:
correlation_df

In [ ]:
# if any of the region names contain the string "_" then remove it
correlation_df["region"] = correlation_df["region"].str.replace("_", " ")

In [ ]:
correlation_df.head()

In [ ]:
EEZ_shapefile["SOVEREIGN1"]

In [ ]:
# Create two new columns in the geopandas dataframe 'EEZ_shapefile'
# 'correlation' - the correlation between the NAO and the offshore wind CFs
# 'p-value' - the p-value of the correlation
EEZ_shapefile["correlation"] = np.nan
EEZ_shapefile["p-value"] = np.nan

In [ ]:
EEZ_shapefile.head()

In [ ]:
# Loop over the regions in correlation_df
for region in correlation_df["region"]:
    # Extract the row from correlation_df
    row = correlation_df[correlation_df["region"] == region]

    # Extract the correlation and p-value
    corr = row["correlation"].values[0]
    p = row["p-value"].values[0]

    # Set the values in the EEZ_shapefile
    EEZ_shapefile.loc[EEZ_shapefile["TERRITORY1"] == region, "correlation"] = corr
    EEZ_shapefile.loc[EEZ_shapefile["TERRITORY1"] == region, "p-value"] = p

In [ ]:
EEZ_shapefile["TERRITORY1"] == "France", "correlation"

In [ ]:
# Extract the list of Terrirories
territories = EEZ_shapefile["TERRITORY1"]

# Convert to a list
territories = list(territories)

# Print the territories
print(territories)

In [ ]:
# Constrain EEZ shapefile to only include the territories in the list
EEZ_shapefile = EEZ_shapefile[EEZ_shapefile["TERRITORY1"].isin(dicts.countries_list)]

In [ ]:
# Print the correlation values for FRance
print(EEZ_shapefile[EEZ_shapefile["SOVEREIGN1"] == "France"]["correlation"])

In [ ]:
# Import cartopy
import cartopy.crs as ccrs

# Now plot the EEZ_shapefile with the correlation as the color
plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
EEZ_shapefile.plot(
    column="correlation", ax=ax, legend=True, cmap="coolwarm", shrink=0.5
)
# Use cartopy to add the coastlines
ax.coastlines()
# Make the colorbar smaller
cbar = ax.get_figure().get_axes()[1]
cbar.set_ylabel("Correlation", fontsize=12)
cbar.tick_params(labelsize=10)

# Constrain to specific bounds
ax.set_xlim(-50, 50)
ax.set_ylim(30, 80)

In [ ]:
# Now plot the EEZ_shapefile with the correlation as the color
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
cax = EEZ_shapefile.plot(
    column="correlation", ax=ax, cmap="coolwarm", add_colorbar=False
)

# Use cartopy to add the coastlines
ax.coastlines()

# Add colorbar
cbar = fig.colorbar(cax.collections[0], ax=ax, shrink=0.5)
cbar.set_label("Correlation")

# Constrain to specific bounds
ax.set_xlim(-50, 50)
ax.set_ylim(30, 80)